In [ ]:
import numpy as np
from functools import reduce
import pandas as pd

In [1]:
from esios import ESIOSClient
client = ESIOSClient('59eafc8896fb679e47acd230a34e72cfec2155dbca4d639e08069e24ff6f9b5b')
endpoint = client.endpoint(name='indicators')

In [2]:
indicators = endpoint.list()

In [3]:
indicators_terciaria = indicators[indicators.description.str.contains('erciaria')]

In [51]:
indicators_terciaria.to_csv("../data/indicators_terciaria.csv")

In [5]:
dfs = []
start = '2023-01-01'
end = '2024-09-09'
for id in list(indicators_terciaria.id):
    print(id)
    df = endpoint.select(id).historical(start=start, end=end)
    dfs.append(df)

535
536
539
540
545
666
667
672
673
674
675
676
677
688
689
690
691
692
693
694
695
696
697
714
715
716
717
732
749
762
767
768
1037
1138
1139
1140
1141
1142
1143
1144
1145
1146
1147
1298
1299
1300
1301
1302
1303
1304
1305
1306
1307
1308
1309
1310
1311
1312
1313
1314
1315
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328
1329
1330
1331
1332
1333
1782
1783
1784
1787
1788
1789
1824
1825
1826
1827
1828
1829
1830
1831
1870
1883
1884
1885
1886
1891
1892
1893
1894
10052
10054
10055
10057
10250
10251
10323
10324
10325
10360
10361
10383
10384
10385
10386
10387
10394
10395
10396
10397
10398
10399
10400
10401


In [15]:
dfs[0]

,535,geo_id,geo_name
datetime,,,
2023-01-01 00:00:00+01:00,0.00,8741,Península
2023-01-01 00:15:00+01:00,0.00,8741,Península
2023-01-01 00:30:00+01:00,0.00,8741,Península
2023-01-01 00:45:00+01:00,0.00,8741,Península
2023-01-01 01:00:00+01:00,0.00,8741,Península
...,...,...,...
2024-09-09 22:45:00+02:00,0.00,8741,Península
2024-09-09 23:00:00+02:00,12.75,8741,Península
2024-09-09 23:15:00+02:00,12.75,8741,Península


In [22]:
import copy
dfs_aux = copy.copy(dfs)

In [32]:
dfs_aux[0].columns[0]

'535'

In [36]:
for df in dfs_aux:
    if df.columns.size > 0:  # Check if the DataFrame has at least one column
        df.to_csv(f'../data/all_terciaria_df/{df.columns[0]}.csv')
    else:
        print("DataFrame is empty, skipping...")

DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFrame is empty, skipping...
DataFram

In [38]:
required_columns = ['geo_id', 'geo_name']

for df in dfs_aux:
    if not df.empty:  # Check if the DataFrame is not empty
        if not all(col in df.columns for col in required_columns):  # Check if all required columns are present
            print("DataFrame missing required columns:")
            print(df)
# todos tienen las columnas requeridas


In [39]:
# Ensure each DataFrame has 'geo_id' and 'geo_name' columns, and 'datetime' is the index name
dfs_non_empty = [df for df in dfs if not df.empty and 'geo_id' in df.columns and 'geo_name' in df.columns]

# Now apply your original merge logic
df_base = reduce(lambda x, y: pd.merge(x.reset_index(), 
                                       y.reset_index(), 
                                       on=['datetime', 'geo_id', 'geo_name'], 
                                       how='outer').set_index('datetime'), 
                 dfs_non_empty)


In [40]:
df_base.to_csv('../data/merged_dfs.csv')


In [46]:
df_base.geo_id.value_counts()

geo_id
8741    177972
3        56392
2        56020
1        55624
8825     53352
Name: count, dtype: int64

In [69]:
df_base = pd.read_csv("../data/merged_dfs.csv")
df = df_base[df_base.geo_name.str.contains("Pen")]

In [85]:
df['datetime'] = pd.to_datetime(df['datetime'], utc=True)

/tmp/ipykernel_6323/1697135093.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['datetime'] = pd.to_datetime(df['datetime'], utc=True)


In [87]:
def filter_round_hours(df, datetime_col):
    # Extract minute and second from the datetime column
    is_start_of_hour = (df[datetime_col].dt.minute == 0) & (df[datetime_col].dt.second == 0)
    return df[is_start_of_hour]

df_00 = filter_round_hours(df,"datetime")

In [89]:
df_00.fillna(0)

/tmp/ipykernel_6323/2635924041.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_00.fillna(0)


,datetime,535,geo_id,geo_name,536,539,540,545,666,667,...,10386,10387,10394,10395,10396,10397,10398,10399,10400,10401
3,2022-12-31 23:00:00+00:00,0.00,8741,Península,0.00,0.0,25.00,20302.0,125.00,0.00,...,0.00,0.000,0.000,0.0,0.0,0.0,0.00,0.000,0.0,0.0
31,2023-01-01 00:00:00+00:00,0.00,8741,Península,17.50,0.0,16.75,19530.0,0.00,300.00,...,0.00,-1540.000,140.000,0.0,0.0,0.0,0.00,-1540.000,0.0,0.0
59,2023-01-01 01:00:00+00:00,0.00,8741,Península,24.00,20.0,0.00,18579.0,0.00,0.00,...,0.00,-660.000,60.000,0.0,0.0,0.0,0.00,-660.000,0.0,0.0
87,2023-01-01 02:00:00+00:00,0.00,8741,Península,42.75,0.0,67.75,17274.0,0.00,674.00,...,0.00,0.000,0.000,0.0,0.0,0.0,0.00,0.000,0.0,0.0
115,2023-01-01 03:00:00+00:00,0.00,8741,Península,26.50,0.0,0.00,16760.0,0.00,848.00,...,0.00,-2848.505,142.425,0.0,0.0,0.0,0.00,-2848.505,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399227,2024-09-09 17:00:00+00:00,0.00,8741,Península,49.25,0.0,53.50,28905.0,19.00,1454.75,...,1102.28,0.000,0.000,22.5,0.0,0.0,1102.28,0.000,0.0,0.0
399255,2024-09-09 18:00:00+00:00,0.00,8741,Península,0.00,0.0,0.00,30486.0,100.00,100.00,...,0.00,11212.500,162.500,0.0,0.0,0.0,0.00,11212.500,0.0,0.0
399283,2024-09-09 19:00:00+00:00,0.00,8741,Península,0.00,0.0,0.00,31445.0,0.00,225.00,...,0.00,0.000,0.000,0.0,0.0,0.0,0.00,0.000,0.0,0.0
399311,2024-09-09 20:00:00+00:00,0.00,8741,Península,32.50,25.0,0.00,29432.0,131.25,684.25,...,12023.25,0.000,0.000,102.5,0.0,0.0,12023.25,0.000,0.0,0.0


In [90]:
df_00.to_csv("../data/by_hour_nan_0.csv")

Now... the magic!!! Trying to predict blabla

In [92]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

2024-09-10 16:56:11.424650: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-10 16:56:11.435287: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-10 16:56:11.437920: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-10 16:56:11.448002: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-10 16:56:12.145079: W tensorflow/compiler/tf2

In [93]:
df_00.set_index('datetime',inplace=True)

KeyError: "None of ['datetime'] are in the columns"

In [97]:
df_nn = df_00.drop(columns=["geo_id","geo_name"])

In [99]:
df_nn = df_nn.fillna(0)

/tmp/ipykernel_6323/447902394.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_nn = df_nn.fillna(0)


In [104]:
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df_nn[:])

In [108]:
scaled_features.shape

(14831, 58)

In [110]:
df_nn.shape

(14831, 58)

In [111]:
df_nn.loc[:] = scaled_features

In [114]:
def create_sequences(df, sequence_length):
    X, y = [], []
    for i in range(len(df) - sequence_length):
        X.append(df.iloc[i:i+sequence_length])
        y.append(df.iloc[i+sequence_length])
    return np.array(X), np.array(y)

In [119]:
sequence_length = 24
X, y = create_sequences(df_nn, sequence_length)

In [123]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [124]:
# Define the model
model = Sequential([
    LSTM(50, activation='relu', input_shape=(sequence_length, X.shape[2])),
    Dense(y.shape)
])

model.compile(optimizer='adam', loss='mse')

# Summary of the model
model.summary()


2024-09-10 17:04:24.316827: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
/home/aner/.venvs/energy/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50)             │        21,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,851 (85.36 KB)

 Trainable params: 21,851 (85.36 KB)

 Non-trainable params: 0 (0.00 B)